In [1]:
import torch
import torch.nn as nn
from strokes import StrokePatientsMIDataset, StrokePatientsMIProcessedDataset
from strokesdict import STROKEPATIENTSMI_LOCATION_DICT
from torcheeg.transforms import Select,BandSignal,Compose
from to import ToGrid, ToTensor
from downsample import SetSamplingRate
from baseline import BaselineCorrection

dataset = StrokePatientsMIDataset(root_path='./subdataset',
                                  io_path='.torcheeg/dataset',
                        chunk_size=500,  # 1 second
                        overlap = 250,
                        offline_transform=Compose(
                                [BaselineCorrection(),
                                SetSamplingRate(origin_sampling_rate=500,target_sampling_rate=128),
                                BandSignal(sampling_rate=128,band_dict={'frequency_range':[8,40]})
                                ]),
                        online_transform=Compose(
                                [ToGrid(STROKEPATIENTSMI_LOCATION_DICT),ToTensor()]),
                        label_transform=Select('label'),
                        num_worker=8
)
print(dataset[0][0].shape) #EEG shape:torch.Size([1, 128, 9, 9])
print(dataset[0][1])  # label (int)
print(len(dataset))

/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[2025-05-15 19:11:12] INFO (torcheeg/MainThread) 🔍 | Processing EEG data. Processed EEG data has been cached to .torcheeg/dataset.
[2025-05-15 19:11:12] INFO (torcheeg/MainThread) ⏳ | Monitoring the detailed processing of a record for debugging. The processing of other records will only be reported in percentage to keep it clean.
[PROCESS]: 100%|██████████| 10/10 [00:00<00:00, 131.93it/s]

[RECORD ./subdataset/sourcedata/sub-41/sub-41_task-motor-imagery_eeg.mat]: 0it [00:00, ?it/s]
[RECORD ./subdataset/sourcedata/sub-41/sub-41_task-motor-imagery_eeg.mat]: 1it [00:00,  4.56it/s]
[RECORD ./subdataset/sourcedata/sub-41/sub-41_task-motor-imagery_eeg.mat]: 2it [00:00,  4.91it/s]
[RECORD ./subdataset/sourcedata/sub-41/sub-41_task-m

torch.Size([1, 128, 9, 9])
0
2400


In [2]:
from eegswintransformer import SwinTransformer

HYPERPARAMETERS = {
    "seed": 42,
    "batch_size": 12,
    "lr": 1e-5,
    "weight_decay": 1e-4,
    "num_epochs": 50,
}
from torcheeg.model_selection import KFoldPerSubjectGroupbyTrial
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from torch.utils.data import DataLoader
from classifier_loss import ClassifierTrainer

k_fold = KFoldPerSubjectGroupbyTrial(
    n_splits=4,
    shuffle=True,
    split_path='.torcheeg/model_selection',
    random_state=42)

training_metrics = []
test_metrics = []

for i, (training_dataset, test_dataset) in enumerate(k_fold.split(dataset)):
    model = SwinTransformer(patch_size=(8,3,3),
                            depths=(2, 6, 4),
                            num_heads=(3,6,8),
                            window_size=(3,3,3)
                            )
    trainer = ClassifierTrainer(model=model,
                                num_classes=2,
                                lr=HYPERPARAMETERS['lr'],
                                weight_decay=HYPERPARAMETERS['weight_decay'],
                                metrics=["accuracy"],
                                accelerator="gpu")
    training_loader = DataLoader(training_dataset,
                            batch_size=HYPERPARAMETERS['batch_size'],
                            shuffle=True)
    test_loader = DataLoader(test_dataset,
                            batch_size=HYPERPARAMETERS['batch_size'],
                            shuffle=False)
    # 提前停止回调
    early_stopping_callback = EarlyStopping(
        monitor='train_loss',
        patience=10,
        mode='min',
        verbose=True
    )
    trainer.fit(training_loader,
                test_loader,
                max_epochs=HYPERPARAMETERS['num_epochs'],
                callbacks=[early_stopping_callback],
                # enable_progress_bar=True,
                enable_model_summary=False,
                limit_val_batches=0.0)
    training_result = trainer.test(training_loader,
                                enable_progress_bar=True,
                                enable_model_summary=True)[0]
    test_result = trainer.test(test_loader,
                            enable_progress_bar=True,
                            enable_model_summary=True)[0]
    training_metrics.append(training_result["test_accuracy"])
    test_metrics.append(test_result["test_accuracy"])
     

[2025-05-15 19:11:27] INFO (torcheeg/MainThread) 📊 | Create the split of train and test set.
[2025-05-15 19:11:27] INFO (torcheeg/MainThread) 😊 | Please set split_path to .torcheeg/model_selection for the next run, if you want to use the same setting for the experiment.
/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, 

Epoch 0: 100%|██████████| 14/14 [00:02<00:00,  6.94it/s, loss=0.0446, train_loss=-0.194, train_accuracy=1.000] 

Metric train_loss improved. New best score: -0.194


Epoch 0: 100%|██████████| 14/14 [00:02<00:00,  6.93it/s, loss=0.0446, train_loss=-0.194, train_accuracy=1.000]

/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/pytorch_lightning/core/module.py:493: UserWarning: You called `self.log('train_loss', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/pytorch_lightning/core/module.py:493: UserWarning: You called `self.log('train_accuracy', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
[2025-05-15 19:11:35] INFO (torcheeg/MainThread) 
[Train] train_loss: 0.045 train_accuracy: 0.625 



...

Testing DataLoader 0: 100%|██████████| 17/17 [00:00<00:00, 73.49it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy                 1.0
        test_loss          0.0009545990615151823
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 4/4 [00:00<00:00, 78.04it/s]

[2025-05-15 19:41:22] INFO (torcheeg/MainThread) 
[Test] test_loss: 0.209 test_accuracy: 0.950 



Testing DataLoader 0: 100%|██████████| 4/4 [00:00<00:00, 72.06it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy          0.949999988079071
        test_loss           0.20926740765571594
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [4]:
import numpy as np

for i in range(0, len(test_metrics), 4):
    print(f"{i}\t"
          f"{np.mean(training_metrics[i:i+4]):.3f}\t"
          f"{np.std(training_metrics[i:i+4]):.3f}\t"
          f"{np.mean(test_metrics[i:i+4]):.3f}\t"
          f"{np.std(test_metrics[i:i+4]):.3f}")

0	1.000	0.000	0.844	0.128
4	1.000	0.000	0.822	0.099
8	1.000	0.000	0.844	0.134
12	1.000	0.000	0.909	0.118
16	1.000	0.000	0.891	0.079
20	1.000	0.000	0.856	0.149
24	1.000	0.000	0.800	0.171
28	1.000	0.000	0.859	0.105
32	1.000	0.000	0.903	0.139
36	1.000	0.000	0.906	0.069


In [5]:
print(test_metrics)
avg = sum(test_metrics) / len(test_metrics)
print(avg)

[0.637499988079071, 0.8374999761581421, 0.949999988079071, 0.949999988079071, 0.675000011920929, 0.8125, 0.8500000238418579, 0.949999988079071, 0.625, 0.8500000238418579, 0.925000011920929, 0.9750000238418579, 0.7124999761581421, 0.925000011920929, 1.0, 1.0, 0.762499988079071, 0.8999999761581421, 0.9750000238418579, 0.925000011920929, 0.625, 0.824999988079071, 0.9750000238418579, 1.0, 0.512499988079071, 0.8374999761581421, 0.949999988079071, 0.8999999761581421, 0.7124999761581421, 0.824999988079071, 0.8999999761581421, 1.0, 0.6625000238418579, 0.9750000238418579, 1.0, 0.9750000238418579, 0.7875000238418579, 0.9375, 0.949999988079071, 0.949999988079071]
0.8634374991059304
